In [47]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import imblearn
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
%matplotlib inline

In [48]:
data=pd.read_csv("D:/my_work/python_codes/git_projects/car_insurance_claim_prediction/dataset/train.csv")
data=data.drop("policy_id",axis=1)

In [49]:
from sklearn.model_selection import train_test_split
train_set,val_set=train_test_split(data,test_size=0.2,stratify=data["is_claim"])

In [50]:
x_train=train_set.drop("is_claim",axis=1)
y_train=train_set["is_claim"]

x_val=val_set.drop("is_claim",axis=1)
y_val=val_set["is_claim"]

Try dropping the first 3 colums a it is already normalised

In [51]:
x_num=x_train.select_dtypes(include="number").columns
x_train_num=x_train[x_num]
x_val_num=x_val[x_num]

scaler=MinMaxScaler()
x_train_num=pd.DataFrame(scaler.fit_transform(x_train_num),columns=x_train_num.columns)
x_val_num=pd.DataFrame(scaler.transform(x_val_num),columns=x_val_num.columns)

In [52]:
x_cat=x_train.select_dtypes(include="object").columns
x_train_cat=pd.DataFrame()
x_val_cat=pd.DataFrame()
enc=LabelEncoder()
for i in x_cat:
    x_train_cat[i]=pd.DataFrame(enc.fit_transform(x_train[i]))
    x_val_cat[i]=pd.DataFrame(enc.transform(x_val[i]))

In [53]:
x_train=pd.concat([x_train_num,x_train_cat],axis=1)
x_val=pd.concat([x_val_num,x_val_cat],axis=1)

In [54]:
import imblearn
im=imblearn.over_sampling.SMOTE(random_state=42)
x_train,y_train = im.fit_resample(x_train,y_train)

In [58]:
from sklearn.model_selection import StratifiedKFold,cross_val_score
from sklearn.metrics import f1_score
clf = XGBClassifier()

k_folds = StratifiedKFold(n_splits = 5)

scores = cross_val_score(clf, x_train, y_train, cv = k_folds)
print(scores)


[0.82535613 0.99094017 0.99071225 0.99071225 0.99162393]


In [62]:
clf.fit(x_train,y_train)
pred=clf.predict(x_val)
print(f1_score(y_val,pred))

0.012594458438287154


In [63]:
from sklearn.model_selection import StratifiedKFold,cross_val_score
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()

k_folds = StratifiedKFold(n_splits = 5)

scores = cross_val_score(clf, x_train, y_train, cv = k_folds)
print(scores)

clf.fit(x_train,y_train)
pred=clf.predict(x_val)
print(f1_score(y_val,pred))


[0.86592593 0.92262108 0.91931624 0.91794872 0.9208547 ]
0.11646136618141097


In [55]:
# num_type=list(x_train.select_dtypes(include="number").columns)
# cat_type=list(x_val.select_dtypes(include="object").columns)

# numeric_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='median')),
#     ('scaler', MinMaxScaler())])

# categorical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
#     ('onehot', LabelEncoder())])

# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numeric_transformer, num_type),
#         ('cat', categorical_transformer, cat_type)]).fit(x_train)

# X_train_pipe = preprocessor.transform(x_train)
# X_test_pipe = preprocessor.transform(x_val)



In [56]:
# log_reg = Pipeline(steps=[('preprocessor', preprocessor),
#                       ('classifier', LogisticRegression(solver='lbfgs'))])

# clf = Pipeline(steps=[('preprocessor', preprocessor),
#                       ('classifier', XGBClassifier())])